In [3]:
!pip install evaluate jiwer sacrebleu rich -q

In [4]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)
import evaluate 
import jiwer
from tqdm import tqdm
import os
import random
from rich.console import Console
from rich.table import Table

In [5]:
class TrainingConfig:

    MODEL_NAME = "t5-base" 
    DATASET_PATH = "/kaggle/input/ocr-post-correction/combined.csv" 
    INPUT_COLUMN = "input_text"  
    TARGET_COLUMN = "output_text" 
    PREFIX = "correct OCR error: " 
    MODEL_OUTPUT_DIR = "./ocr_correction_model" 
    
    # Training hyperparameters
    LEARNING_RATE = 2e-5
    NUM_EPOCHS = 10 
    BATCH_SIZE = 16 
    WEIGHT_DECAY = 0.01
    LOGGING_STEPS = 100
    SAVE_STEPS = 500

In [ ]:
def main():
    """
    Main function to run the entire training and evaluation pipeline.
    """
    config = TrainingConfig()

    try:
        df = pd.read_csv(config.DATASET_PATH)
        
        if config.INPUT_COLUMN not in df.columns or config.TARGET_COLUMN not in df.columns:
            print(f"Warning: Columns '{config.INPUT_COLUMN}' and '{config.TARGET_COLUMN}' not found.")
            df.columns = [config.INPUT_COLUMN, config.TARGET_COLUMN]
            print(f"Assigned column names: {df.columns.tolist()}")

        dataset = Dataset.from_pandas(df)
        dataset = dataset.train_test_split(test_size=0.1)
        
        # print(f"Dataset loaded. Training examples: {len(dataset['train'])}, Validation examples: {len(dataset['test'])}")
        # print("Sample data:", dataset['train'][0])

    except FileNotFoundError:
        print(f"Error: The file '{config.DATASET_PATH}' was not found.")
        return

    tokenizer = AutoTokenizer.from_pretrained(config.MODEL_NAME)

    def preprocess_function(examples):
        # adding prefix on each tokens for our custom OCR task
        inputs = [config.PREFIX + doc for doc in examples[config.INPUT_COLUMN]]
        model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
        
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(examples[config.TARGET_COLUMN], max_length=128, truncation=True, padding="max_length")
        
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    tokenized_datasets = dataset.map(preprocess_function, batched=True)

    
    # Fine-tuning the T5-Base Seq2Seq Model
    model = AutoModelForSeq2SeqLM.from_pretrained(config.MODEL_NAME)
    
    training_args = Seq2SeqTrainingArguments(
        output_dir=config.MODEL_OUTPUT_DIR,
        num_train_epochs=config.NUM_EPOCHS,
        per_device_train_batch_size=config.BATCH_SIZE,
        per_device_eval_batch_size=config.BATCH_SIZE,
        learning_rate=config.LEARNING_RATE,
        weight_decay=config.WEIGHT_DECAY,
        eval_strategy="epoch",
        save_strategy="epoch",
        logging_steps=config.LOGGING_STEPS,
        predict_with_generate=True,
        fp16=torch.cuda.is_available(),
        push_to_hub=False,
        save_total_limit=2,
        report_to="none",
    )
    
    data_collator = DataCollatorForSeq2Seq(
        tokenizer=tokenizer, 
        model=model
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["test"],
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    trainer.train()

    trainer.save_model(config.MODEL_OUTPUT_DIR)
    tokenizer.save_pretrained(config.MODEL_OUTPUT_DIR)

if __name__ == "__main__":
    main()

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Map:   0%|          | 0/15820 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


...

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/tmp/ipykernel_35/3218032789.py:70: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.400900,0.310984
2,0.330900,0.261883
3,0.293900,0.235851
4,0.272600,0.219290
5,0.256400,0.206588
6,0.245000,0.197185
7,0.237200,0.191480
8,0.230700,0.186337
9,0.227900,0.184093
10,0.227500,0.183071


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked t

In [7]:
# !zip -r ocr_correction_model.zip /kaggle/working/ocr_correction_model

In [8]:
!pip install evaluate jiwer sacrebleu rich -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
class EvalConfig:
    MODEL_DIR = "/kaggle/working/ocr_correction_model"
    DATASET_PATH = "/kaggle/input/ocr-post-correction/combined.csv" 
    INPUT_COLUMN = "input_text"
    TARGET_COLUMN = "output_text"
    PREFIX = "correct OCR error: "
    BATCH_SIZE = 16

def main_refined_evaluation():
    config = EvalConfig()
    console = Console()
    
    with console.status("[bold green]Loading model and tokenizer...") as status:
        try:
            model = AutoModelForSeq2SeqLM.from_pretrained(config.MODEL_DIR)
            tokenizer = AutoTokenizer.from_pretrained(config.MODEL_DIR)
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model.to(device)
            console.log("Model loaded successfully.")
        except OSError:
            console.print(f"Error: Model not found at '{config.MODEL_DIR}'. Please run the training cell first.", style="bold red")
            return

    with console.status("[bold green]Loading and preparing dataset...") as status:
        try:
            full_df = pd.read_csv(config.DATASET_PATH, dtype=str).fillna('')
            test_df = full_df.sample(frac=0.1, random_state=42)
            ocr_texts = test_df[config.INPUT_COLUMN].tolist()
            ground_truth_texts = test_df[config.TARGET_COLUMN].tolist()
        except FileNotFoundError:
            console.print(f"Error: Dataset not found at '{config.DATASET_PATH}'.", style="bold red")
            return
        except KeyError as e:
            console.print(f"Error: Column {e} not found in the dataset.", style="bold red")
            return

    predictions = []
    
    for i in tqdm(range(0, len(ocr_texts), config.BATCH_SIZE), desc="Predicting", leave=False):
        batch_texts = ocr_texts[i:i + config.BATCH_SIZE]
        prefixed_batch = [config.PREFIX + text for text in batch_texts]
        
        inputs = tokenizer(prefixed_batch, return_tensors="pt", padding=True, truncation=True, max_length=128)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        with torch.no_grad():
            output_ids = model.generate(**inputs, max_length=128, num_beams=5, early_stopping=True)
        
        batch_preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
        predictions.extend(batch_preds)

    with console.status("[bold green]Calculating evaluation metrics...") as status:
        cer_score = jiwer.cer(ground_truth_texts, predictions)
        wer_score = jiwer.wer(ground_truth_texts, predictions)
        bleu_metric = evaluate.load("sacrebleu")
        bleu_score = bleu_metric.compute(predictions=predictions, references=[[ref] for ref in ground_truth_texts])
        exact_matches = sum(1 for pred, ref in zip(predictions, ground_truth_texts) if pred.strip() == ref.strip())
        em_score = exact_matches / len(predictions)

    table = Table(title="Model Evaluation Report", show_header=True, header_style="bold magenta")
    table.add_column("Metric", style="dim", width=25)
    table.add_column("Score", justify="right")
 
    table.add_row("Character Error Rate (CER)", f"{cer_score:.4f}")
    table.add_row("Word Error Rate (WER)", f"{wer_score:.4f}")
    table.add_row("BLEU Score", f"{bleu_score['score']:.2f})")
    table.add_row("Exact Match (EM %)", f"{em_score*100:.2f}%")
    
    console.print(table)
    
    console.print("\ Random Sample Predictions ---", style="bold yellow")
    
    random_indices = random.sample(range(len(predictions)), 3)
    
    for i, index in enumerate(random_indices):
        
        sample_table = Table(title=f"Sample {i+1}", show_header=False, box=None, padding=(0,1))
        sample_table.add_column(width=15)
        sample_table.add_column()
        sample_table.add_row("[bold]Input[/bold]", f": '{ocr_texts[index]}'")
        sample_table.add_row("[bold green]Ground Truth[/bold green]", f": '{ground_truth_texts[index]}'")
        sample_table.add_row("[bold cyan]Model Output[/bold cyan]", f": '{predictions[index]}'")
        console.print(sample_table)
        
main_refined_evaluation()


Output()

[16:55:03] Model loaded successfully.                                                              ]8;id=681453;file:///tmp/ipykernel_35/1029302692.py\1029302692.py]8;;\:]8;id=735392;file:///tmp/ipykernel_35/1029302692.py#19\19]8;;\

Output()

Output()

       Model Evaluation Report        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Metric                    ┃  Score ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ Character Error Rate      │ 0.0296 │
│ (CER)                     │        │
│ Word Error Rate (WER)     │ 0.1449 │
│ BLEU Score                │ 81.19) │
│ Exact Match (EM %)        │  1.71% │
└───────────────────────────┴────────┘

\ Random Sample Predictions ---

                                                     Sample 1                                                      
 Input            : 'ID: 5Z377S73, Name: Lokendra8ir Saam, DOB: Date: %2004/04/29, Gender: Mal3, District: Rolqa,  
                  Municipality: Karnalbazar, Father's name: Sugam  Kumar Chavhan, Mother's name: Ahima L@ta        
                  Paudel'                                                                                          
 Ground Truth     : 'ID: 52377573, Name: Lokendra Bir Saam, DOB: 2004-04-29, Gender: Male, District: Rolpa,        
                  Municipality: Kamalbazar, Father's Name: Sugam Kumar Chauhan, Mother's Name: Anima Lata Paudel'  
 Model Output     : 'ID: 52377573, Name: Lokendrashar Saam, DOB: 2004-04-29, Gender: Male, District: Rolpa,        
                  Municipality: Karnalbazar, Father's Name: Sugam Kumar Chauhan, Mother's Name: Ahima Lata Paudel'

                                                     Sample 2                                                      
 Input            : 'ID: 608644424, Name: RobiRajbansh#i, DOB: Date: 200/11/_11, Gender: Mal~e, District: Huml@,   
                  Municipality: K@urkot, Father's name: Nirol NarsinghChamlagain, Mother's name: Aleera            
                  Priyanza1a~ Halc laliya'                                                                         
 Ground Truth     : 'ID: 60864424, Name: Robi Rajbanshi, DOB: 2004-11-11, Gender: Male, District: Humla,           
                  Municipality: Kapurkot, Father's Name: Nirol Narsingh Chamlagain, Mother's Name: Aleena          
                  Priyanzala Haldaliya'                                                                            
 Model Output     : 'ID: 60864424, Name: Robi Rajbanshi, DOB: 2000-11-11, Gender: Male, District: Humla,           
                  Municipality: Kurkot, Father's Name: Nirol Narsingh Chamlagain, Mother's Name: Aleera Priyanzala 
                  Haldaliya'

                                                     Sample 3                                                      
 Input            : 'ID: 69A19658, Name: Ruchika Prabha Du1al, DOB: DOB: 18-02-1987, Gender: Fen nale, District:   
                  Su,khet, Municipality: Sabh@pokh%ari, Father's name: NitesDom, Mother's name: hijaLha@mu Rawal'  
 Ground Truth     : 'ID: 69419658, Name: Ruchika Prabha Dulal, DOB: 1987-02-18, Gender: Female, District: Surkhet, 
                  Municipality: Sabhapokhari, Father's Name: Nites Dom, Mother's Name: Shija Lhamu Rawal'          
 Model Output     : 'ID: 69419658, Name: Ruchika Prabha Dulaal, DOB: 1987-02-18, Gender: Female, District:         
                  Sindhuli, Municipality: Sabhapokhari, Father's Name: Nites Dom, Mother's Name: Hija Lhamu Rawal'